In [40]:
import pandas as pd
import os
from sqlalchemy import create_engine

In [51]:
# define the file path of the .csv
filepath = '../Datafiniti_Fast_Food_Restaurants_May19.csv'

# read csv into pandas and create a dataframe
fastfood_df = pd.read_csv(filepath)

In [52]:
# dropping null value columns to avoid errors 
fastfood_df.dropna(inplace=True)

In [53]:
# display the first 5 rows
fastfood_df.head()

,id,dateAdded,dateUpdated,address,categories,primaryCategories,city,country,keys,latitude,longitude,name,postalCode,province,sourceURLs,websites
0,AWrSh_KgsVYjT2BJAzaH,2019-05-19T23:58:05Z,2019-05-19T23:58:05Z,2555 11th Avenue,"Fast Food Restaurants,Hamburgers and Hot Dogs,...",Accommodation & Food Services,Greeley,US,us/co/greeley/255511thavenue/554191587,40.39629,-104.69699,Carl's Jr.,80631,CO,https://www.yellowpages.com/greeley-co/mip/car...,https://www.carlsjr.com/?utm_source=Yextandutm...
1,AWEKlA-LIxWefVJwxG9B,2018-01-18T18:30:23Z,2019-05-19T23:45:05Z,2513 Highway 6 And 50,"Restaurant,Mexican Restaurants,Fast Food Resta...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/2513highway6and50/1550891556,39.08135,-108.58689,Del Taco,81505,CO,http://www.citysearch.com/profile/772076870/gr...,http://www.deltaco.com
2,AWrSfAcYsVYjT2BJAzPt,2019-05-19T23:45:04Z,2019-05-19T23:45:04Z,1125 Patterson Road,"Sandwich Shops,Fast Food Restaurants,Restauran...",Accommodation & Food Services,Grand Junction,US,us/co/grandjunction/1125pattersonroad/-2137447852,39.09148,-108.55411,Which Wich,81506,CO,https://www.yellowpages.com/grand-junction-co/...,http://www.whichwich.com
3,AWrSa3NAQTFama1Xpkbz,2019-05-19T23:26:58Z,2019-05-19T23:26:58Z,3455 N Salida Court,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Aurora,US,us/co/aurora/3455nsalidacourt/1143321601,39.76369,-104.77671,Chipotle Mexican Grill,80011,CO,https://www.yellowpages.com/aurora-co/mip/chip...,http://www.chipotle.com
4,AWrSaVGzZ4Yw-wtdgcaB,2019-05-19T23:24:38Z,2019-05-19T23:24:38Z,5225 E Colfax Avenue,"Fast Food Restaurants,Mexican Restaurants,Rest...",Accommodation & Food Services,Denver,US,us/co/denver/5225ecolfaxavenue/-864103396,39.74044,-104.92636,Taco Bell,80220,CO,https://www.yellowpages.com/denver-co/mip/taco...,https://locations.tacobell.com/co/denver/5225-...


In [45]:
fastfood_cols_df['Restaurant'].value_counts()

NameError: name 'fastfood_cols_df' is not defined

In [54]:
fastfood_cols_df2 = fastfood_df[['city', 'name']]

# rename the columns
fastfood_cols_df2.columns = ['City', 'Restaurant']
fastfood_cols_df2

,City,Restaurant
0,Greeley,Carl's Jr.
1,Grand Junction,Del Taco
2,Grand Junction,Which Wich
3,Aurora,Chipotle Mexican Grill
4,Denver,Taco Bell
...,...,...
9995,Bessemer,Church's Chicken Of Birmingham
9996,Raleigh,Luxury Box
9997,Plover,McDonald's
9998,Hendersonville,Pizza Hut


In [55]:
fastfood_df = fastfood_cols_df2.groupby(['City']).count()
fastfood_df = fastfood_df.sort_values('Restaurant', ascending = False)
fastfood_df = fastfood_df.reset_index()
fastfood_df.columns = ['city', 'number_fastfood']
fastfood_df

,city,number_fastfood
0,Columbus,105
1,Indianapolis,94
2,Birmingham,69
3,Cincinnati,60
4,Dayton,59
...,...,...
2782,Noble,1
2783,Flagler,1
2784,Nitro,1
2785,Nine Mile Falls,1


In [33]:
connection_string = "postgres:postgres@localhost:5432/ucsd_teamproject_ETL"
engine = create_engine(f'postgresql://{connection_string}')

In [56]:
engine.table_names()

['fastfood']

In [37]:
fastfood_df.to_sql(name='fastfood', con=engine, if_exists='append', index=False)

In [18]:
# I wanted to look at the dates when these restaurants were updated to see the spread of years
# I needed to split the string in the 'dateUpdated' column
date = fastfood_df['dateUpdated'].str.split('T', n=1, expand=True)
date[0].value_counts()

2019-04-01    567
2019-05-15    390
2019-05-12    383
2019-05-17    365
2019-04-18    341
             ... 
2019-01-08      1
2018-12-05      1
2019-01-02      1
2019-02-25      1
2019-01-07      1
Name: 0, Length: 144, dtype: int64

In [19]:
# I then added the data string onto the dataframe by creating a new column of the first list with the dates only
fastfood_df['Date'] = date[0]

In [48]:
# filter for only McDonald's fastfood restaurants
# isolate the 'Restaurants' with a conditional and store in a boolean variable
_mcds = fastfood_cols_df['Restaurant'] == "McDonald's"

# subset by applying the boolean variable to the dataframe. this will act as a filter
mcdonalds_df = fastfood_cols_df[_mcds]
# mcdonalds_df['Country'].value_counts()

In [60]:
mcdonalds_index_df = mcdonalds_df.groupby(['City']).count()
mcds = mcdonalds_index_df['Restaurant']
mcds_df = mcds.to_frame()
mcds_df

,Restaurant
City,
Abbeville,1
Aberdeen,1
Abingdon,1
Adairsville,1
Albany,1
...,...
York,2
Yuba City,1
Yukon,1


In [38]:
mcds_df.sort_values('Restaurant', ascending = False)

NameError: name 'mcds_df' is not defined

In [42]:
mcdonalds_df.groupby(['City']).count()

,Country,Restaurant,Date
City,,,
Abbeville,1,1,1
Aberdeen,1,1,1
Abingdon,1,1,1
Adairsville,1,1,1
Albany,1,1,1
...,...,...,...
York,2,2,2
Yuba City,1,1,1
Yukon,1,1,1


In [44]:
# pull the columns we want to use for our dataframe using double brackets to pull the dataframes [[]]
fastfood_cols_df = fastfood_df[['city', 'country', 'name', 'Date']]

# rename the columns
fastfood_cols_df.columns = ['City', 'Country', 'Restaurant', 'Date']
fastfood_cols_df

KeyError: "['Date'] not in index"